<a href="https://colab.research.google.com/github/Siddharthg97/Text-generation-comments/blob/main/Text_gen_few_shot_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os
import re

In [4]:
### GPT2

In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel,BertTokenizer
import torch


In [ ]:
We shall be going to work on text completion using decoder based model - GPT-2 , Falcon and encoder & decoder based T5 & BART

In [ ]:
Example 1:
Input: Mismatch Date of Birth (DOB) is good, customer date of birth is 28/10/1971 from MDM and watchlist date of birth is 00/00/1970 from Refinitv.Mismatch ID,customer ID is 686866 from MDM and watchlist ID is 14557408 from Refinitv.Match Full Legal Name, customer name is ZAW MIN  obtained from MDM and watchlist name is ZAW MIN obtained from Refinitv .Mismatch profile/occupation is good,customer is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, MANUFACTURING sector while watch person as a Director General of Prison Department of Ministry of Home Affairs. Mismatch gender. Mismatch Nationality
Output: Mismatch Date of birth and profile/occupation.Customer date of birth is 28/10/1971 and is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, while watch person  has date of birth is 00/00/1970 and is a Director General of Prison Department of Ministry of Home Affairs, appointed by the State Administration Council (SAC).

Example 2:
Input: Mismatch Date of Birth (DOB), customer date of birth is 20/08/1959 from MDM and watchlist date of birth is 00/00/1958 from Refinitv.Mismatch ID NRIC,customer NRIC is 590820016183 from
MDM and watchlist ID is 9705 from Refinitv.Mismatch Full Legal Name is good,Word count difference as customer name is JAMAL BIN SALLEH obtained
from MDM and watchlist name is JAMAL TAL ABDUL SALEH  MOHD SALEH obtained from Refinitv and Name not sounding similiar as, customer name is
JAMAL BIN SALLEH obtained from MDM and watchlist name is JAMAL TAL ABDUL SALEH obtained from Refinitv.Mismatch nationality is good.Mismatch
profile/occupation MANUFACTURING LABOURERS AT GENTING MALAYSIA BERHAD, GAMBLING/BETTING ACTIVITIES/JUNKET /CASINO/GAMING HOUSE SECTOR while watchlist is MEMBER OF
THE BOARD OF DIRECTORS OF GUANGXI GUIGANG FINANCIAL INVESTMENT AND DEVELOPMENT GROUP CO LTD (SOE) (JAN 2021 - ).

Output: Mismatch name Word count difference , Name not sounding similiar and mismatch nationality.Customer name is JAMAL BIN SALLEH and nationality
is Indian where as watch person name is JAMAL TAL ABDUL SALEH  MOHD SALEH and nationality is African

Example 3:
Input: Mismatch Date of Birth (DOB),customer date of birth is 20/08/1981 from MDM and watchlist date of birth is 04/08/1970 from Refinitv.
Mismatch ID passport is good,customer ID is K3995661 from MDM and watchlist ID is AIDPV3575A from Refinitv.
Match Full Legal Name is good, customer name is SUNIL KUMAR obtained from MDM and watchlist name is SUNIL KUMAR obtained from Refinitv.Mismatch profile/occupation is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ARAB MONETARY FUND ABU DHABI, FINANCIAL INSTITUITION SECTOR while watchperson isDIRECTOR OF COOPERATION AT THE MINISTRY OF FINANCE ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE ASIAN INFRASTRUCTURE DEVELOPMENT BANK (AIIB) ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE INTERNATIONAL BANK
FOR RECONSTRUCTION AND DEVELOPMENT (IBRD) (DEC 2020 - ). ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF INTERNATIONAL FINANCE CORPORATION (IFC) (DEC 2020 - ). ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE INTERNATIONAL DEVELOPMENT ASSOCIATION (IDA) (DEC 2020 - ). ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF MULTILATERAL INVESTMENT GUARANTEE AGENCY (MIGA) (DEC 2020 - ). ALTERNATE REPRESENTATIVE OF THE ADMINISTRATIVE COUNCIL OF INTERNATIONAL CENTRE FOR SETTLEMENT OF INVESTMENT DISPUTES (ICSID) (DEC 2020 - ). MEMBER OF THE BOARD OF EXECUTIVE DIRECTORS OF ARAB MONETARY FUND (NOV 2020 - ).

Output:Mismatch ID passport and occupation. customer has id K3995661 and works as MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ARAB MONETARY FUND ABU DHABI, FINANCIAL INSTITUITION SECTOR while watchperson is DIRECTOR OF COOPERATION AT THE MINISTRY OF FINANCE ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE ASIAN INFRASTRUCTURE DEVELOPMENT BANK (AIIB) ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE INTERNATIONAL BANK FOR RECONSTRUCTION AND DEVELOPMENT (IBRD) (DEC 2020 - ).
as while watchlist ID is AIDPV3575A and works as  DIRECTOR OF COOPERATION AT THE MINISTRY OF FINANCE ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE ASIAN INFRASTRUCTURE DEVELOPMENT BANK (AIIB)
ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE INTERNATIONAL BANK FOR RECONSTRUCTION AND DEVELOPMENT (IBRD) (DEC 2020 - ). ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF INTERNATIONAL FINANCE CORPORATION (IFC) (DEC 2020 - ). ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE INTERNATIONAL DEVELOPMENT ASSOCIATION (IDA) (DEC 2020 - ). ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF MULTILATERAL INVESTMENT GUARANTEE AGENCY (MIGA) (DEC 2020 - ). ALTERNATE REPRESENTATIVE OF THE ADMINISTRATIVE COUNCIL OF INTERNATIONAL CENTRE FOR SETTLEMENT OF INVESTMENT DISPUTES (ICSID) (DEC 2020 - ). MEMBER OF THE BOARD OF EXECUTIVE DIRECTORS OF ARAB MONETARY FUND (NOV 2020 - ).



Now, continue the following text:
Input: Match DOB is good,customer date of birth is 25/06/1973 from MDM and watchlist date of birth is 25/06/1973 from Refinitv.Match ID NRIC,customer NRIC is 730625145117 from MDM and watchlist NRIC is 730625145117 from Refinitv.Match occupation/profile is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR. while watch person is DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD
Output: Match DOB and occupation.Customer date of birth 25/06/1973 and works as MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD while watchlist date of birth is 25/06/1973 and works as DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD



Input:Match DOB is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/02/1988 from Refinitv.Match ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715069 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.Match occupation is good, customer works as MANAGING DIRECTORS AND CHIEF EXECUTIVES AT FORTE INSURANCE (CAMBODIA) PLC, FINANCIAL AND INSURANCE/ TAKAFUL ACTIVITIES SECTOR	MANAGING DIRECTORS AND CHIEF EXECUTIVES AT FORTE INSURANCE (CAMBODIA) PLC, FINANCIAL AND INSURANCE/ TAKAFUL ACTIVITIES SECTOR. MEMBER OF THE BOARD OF DIRECTORS OF CAMBODIAN while watch person works as MEMBER OF THE BOARD OF DIRECTORS OF CAMBODIAN REINSURANCE COMPANY (SOE) (2019 - ). GROUP CEO OF FORTE INSURANCE PLC (CAMBODIA).
Output: Match ID Nric and date of birth.Customer dob is 22/02/1988 and nric id is NRIC is 531211715069 while watchlist date of birth is 22/02/1988 and nric is 531211715069



2528

In [2]:
prompt = """
Example 1:
Input: Mismatch Date of Birth (DOB) is good, customer date of birth is 28/10/1971 from MDM and watchlist date of birth is 00/00/1970 from Refinitv.Mismatch ID,customer ID is 686866 from MDM and watchlist ID is 14557408 from Refinitv.Match Full Legal Name, customer name is ZAW MIN  obtained from MDM and watchlist name is ZAW MIN obtained from Refinitv .Mismatch profile/occupation is good,customer is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, MANUFACTURING sector while watch person as a Director General of Prison Department of Ministry of Home Affairs. Mismatch gender. Mismatch Nationality
Output: Mismatch Date of birth and profile/occupation.Customer date of birth is 28/10/1971 and is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, while watch person  has date of birth is 00/00/1970 and is a Director General of Prison Department of Ministry of Home Affairs, appointed by the State Administration Council (SAC).

Example 2:
Input: Match DOB is good,customer date of birth is 25/06/1973 from MDM and watchlist date of birth is 25/06/1973 from Refinitv.Match ID NRIC,customer NRIC is 730625145117 from MDM and watchlist NRIC is 730625145117 from Refinitv.Match occupation/profile is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR. while watch person is DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD
Output: Match DOB and occupation.Customer date of birth 25/06/1973 and works as MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD while watchlist date of birth is 25/06/1973 and works as DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD


Example 3:
Input: Match DOB is good,customer date of birth is 25/06/1973 from MDM and watchlist date of birth is25/06/1973 from Refinitv.Match ID NRIC,customer NRIC is 730625145117 from MDM and watchlist NRIC is 730625145117 from Refinitv.Match occupation/profile is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR. while watch person is DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD
Output: Match DOB and occupation.Customer date of birth 25/06/1973 and works as MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD while watchlist date of birth is 25/06/1973 and works as DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD


Input:Mismatch DOB is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/06/1988 from Refinitv.Mismatch ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715068 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.
Output:
"""

In [8]:
model_name = 'gpt2'  # You can also use 'gpt2-medium', 'gpt2-large', or 'gpt2-xl'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
model = GPT2LMHeadModel.from_pretrained(model_name)
# model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
# Encode the prompt
inputs = tokenizer(prompt, max_length=900,return_tensors='pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [10]:
print(tokenizer.convert_ids_to_tokens(inputs.input_ids[0]))

['Ċ', 'Example', 'Ġ1', ':', 'Ċ', 'Input', ':', 'ĠM', 'ism', 'atch', 'ĠDate', 'Ġof', 'ĠBirth', 'Ġ(', 'D', 'OB', ')', 'Ġis', 'Ġgood', ',', 'Ġcustomer', 'Ġdate', 'Ġof', 'Ġbirth', 'Ġis', 'Ġ28', '/', '10', '/', '1971', 'Ġfrom', 'ĠMD', 'M', 'Ġand', 'Ġwatch', 'list', 'Ġdate', 'Ġof', 'Ġbirth', 'Ġis', 'Ġ00', '/', '00', '/', '1970', 'Ġfrom', 'ĠRef', 'init', 'v', '.', 'M', 'ism', 'atch', 'ĠID', ',', 'custom', 'er', 'ĠID', 'Ġis', 'Ġ68', '68', '66', 'Ġfrom', 'ĠMD', 'M', 'Ġand', 'Ġwatch', 'list', 'ĠID', 'Ġis', 'Ġ14', '557', '408', 'Ġfrom', 'ĠRef', 'init', 'v', '.', 'Match', 'ĠFull', 'ĠLegal', 'ĠName', ',', 'Ġcustomer', 'Ġname', 'Ġis', 'ĠZ', 'AW', 'ĠMIN', 'Ġ', 'Ġobtained', 'Ġfrom', 'ĠMD', 'M', 'Ġand', 'Ġwatch', 'list', 'Ġname', 'Ġis', 'ĠZ', 'AW', 'ĠMIN', 'Ġobtained', 'Ġfrom', 'ĠRef', 'init', 'v', 'Ġ.', 'M', 'ism', 'atch', 'Ġprofile', '/', 'occup', 'ation', 'Ġis', 'Ġgood', ',', 'custom', 'er', 'Ġis', 'Ġa', 'ĠMAN', 'UF', 'ACT', 'UR', 'ING', 'ĠL', 'AB', 'OUR', 'ERS', 'Ġat', 'ĠBE', 'Y', 'ON', 'ICS', 'ĠPR

In [12]:
# outputs = model.generate(
#         inputs.input_ids,
#         max_length=150,  # Maximum length of the generated text
#         num_beams=5,     # Beam search for better quality
#         no_repeat_ngram_size=2,  # Prevent repetition
#         early_stopping=True,
#         pad_token_id=tokenizer.eos_token_id
#     )
with torch.no_grad():
     outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=100,  # Maximum number of new tokens to generate
        num_beams=5,        # Beam search for better quality
        no_repeat_ngram_size=2,  # Prevent repetition
        early_stopping=True,
        temperature= 0.2,
        pad_token_id=tokenizer.eos_token_id
    )

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [13]:
# outputs[0]

In [16]:
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)


In [17]:
generated_text

'\nExample 1:\nInput: Mismatch Date of Birth (DOB) is good, customer date of birth is 28/10/1971 from MDM and watchlist date of birth is 00/00/1970 from Refinitv.Mismatch ID,customer ID is 686866 from MDM and watchlist ID is 14557408 from Refinitv.Match Full Legal Name, customer name is ZAW MIN  obtained from MDM and watchlist name is ZAW MIN obtained from Refinitv.Mismatch profile/occupation is good,customer is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, MANUFACTURING sector while watch person as a Director General of Prison Department of Ministry of Home Affairs. Mismatch gender. Mismatch Nationality\nOutput: Mismatch Date of birth and profile/occupation.Customer date of birth is 28/10/1971 and is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, while watch person  has date of birth is 00/00/1970 and is a Director General of Prison Department of Ministry of Home Affairs, appointed by the State Administration Council (SAC).\n\nExample 2:\nInput: Match DOB 

In [ ]:
# Input:Mismatch DOB is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/06/1988 from Refinitv.Mismatch ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715068 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.


In [19]:
#Falcon - cannot load model in notebook

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the Falcon model and tokenizer
model_name = 'falcon-model-name'  # Replace with the actual model name
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
inputs = tokenizer(prompt, return_tensors='pt') # max tokens is 2048
# Generate the continuation
with torch.no_grad():
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=100,  # Maximum number of new tokens to generate
        num_beams=5,        # Beam search for better quality
        no_repeat_ngram_size=2,  # Prevent repetition
        early_stopping=True
    )

# Decode the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

NameError: name 'tokenizer' is not defined

In [3]:
#T5 - we need to reduce prompt size more certainly otherwise

In [11]:
from transformers import T5Tokenizer,T5ForConditionalGeneration
model_name = 't5-large'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
# Encode the prompt
input_ids = tokenizer(prompt,max_length=900,return_tensors='pt').input_ids

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [10]:
print(tokenizer.convert_ids_to_tokens(input_ids[0]))

['▁Example', '▁1', ':', '▁In', 'put', ':', '▁Mis', 'match', '▁Date', '▁of', '▁Birth', '▁(', 'DO', 'B', ')', '▁is', '▁good', ',', '▁customer', '▁date', '▁of', '▁birth', '▁is', '▁28', '/10', '/19', '71', '▁from', '▁M', 'DM', '▁and', '▁watch', 'list', '▁date', '▁of', '▁birth', '▁is', '▁', '00', '/', '00', '/19', '70', '▁from', '▁Re', 'fini', 't', 'v', '.', 'M', 'is', 'match', '▁ID', ',', 'custom', 'er', '▁ID', '▁is', '▁6', '86', '866', '▁from', '▁M', 'DM', '▁and', '▁watch', 'list', '▁ID', '▁is', '▁14', '55', '74', '08', '▁from', '▁Re', 'fini', 't', 'v', '.', 'M', 'atch', '▁Full', '▁Legal', '▁Name', ',', '▁customer', '▁name', '▁is', '▁Z', 'AW', '▁', 'MIN', '▁obtained', '▁from', '▁M', 'DM', '▁and', '▁watch', 'list', '▁name', '▁is', '▁Z', 'AW', '▁', 'MIN', '▁obtained', '▁from', '▁Re', 'fini', 't', 'v', '▁', '.', 'M', 'is', 'match', '▁profile', '/', 'occupation', '▁is', '▁good', ',', 'custom', 'er', '▁is', '▁', 'a', '▁', 'MAN', 'UF', 'ACT', 'UR', 'ING', '▁', 'LAB', 'OUR', 'ERS', '▁at', '▁BE',

In [ ]:
# Generate the continuation
with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=100,  # Maximum number of new tokens to generate
        num_beams=5,        # Beam search for better quality
        no_repeat_ngram_size=2,  # Prevent repetition
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )


In [ ]:
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
generated_text

completion_start = generated_text.find("Input:Mismatch DOB is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/06/1988 from Refinitv.Mismatch ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715068 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.\nOutput:") + len("Input:Mismatch DOB is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/06/1988 from Refinitv.Mismatch ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715068 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.\nOutput:")
completion = generated_text[completion_start:].strip()
print(generated_text)
print(completion)

In [ ]:
Input:Mismatch DOB is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/06/1988 from Refinitv.Mismatch ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715068 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.
Output:

In [20]:
#FlanT5

In [13]:
### BART

In [3]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# Encode the prompt
input_ids = tokenizer(prompt,max_length=900,return_tensors='pt').input_ids

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [5]:
# Generate text completion - max tokens
with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=100,  # Maximum number of new tokens to generate
        num_beams=5,        # Beam search for better quality
        no_repeat_ngram_size=2,  # Prevent repetition
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )


# Decode the generated text
completed_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [6]:
completed_text

'Output: Match DOB and occupation.Customer date of birth is 28/10/1971 and is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, while watch person  is 00/00/1970.Match occupation/profile is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR'

In [26]:
prompt

'\nExample 1:\nInput: Mismatch Date of Birth (DOB) is good, customer date of birth is 28/10/1971 from MDM and watchlist date of birth is 00/00/1970 from Refinitv.Mismatch ID,customer ID is 686866 from MDM and watchlist ID is 14557408 from Refinitv.Match Full Legal Name, customer name is ZAW MIN  obtained from MDM and watchlist name is ZAW MIN obtained from Refinitv .Mismatch profile/occupation is good,customer is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, MANUFACTURING sector while watch person as a Director General of Prison Department of Ministry of Home Affairs. Mismatch gender. Mismatch Nationality\nOutput: Mismatch Date of birth and profile/occupation.Customer date of birth is 28/10/1971 and is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, while watch person  has date of birth is 00/00/1970 and is a Director General of Prison Department of Ministry of Home Affairs, appointed by the State Administration Council (SAC).\n\nExample 2:\nInput: Match DOB

In [ ]:
Input:Mismatch DOB is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/06/1988 from Refinitv.Mismatch ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715068 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.
Output:

In [8]:
# Generate the continuation - max length
with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_length=150,  # Maximum length of the generated text
        num_beams=5,     # Beam search for better quality
        no_repeat_ngram_size=2,  # Prevent repetition
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )


# with torch.no_grad():
#     outputs = model.generate(
#           input, max_length=100, num_return_sequences=1,
#           pad_token_id=tokenizer.eos_token_id

#     )

In [9]:

# Decode the generated text
completed_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [10]:
completed_text

'Output: Match DOB and occupation.Customer date of birth is 28/10/1971 and is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, while watch person  is 00/00/1970.Match occupation/profile is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR. Mismatch Full Legal Name.Example 1:¯¯¯¯¯¯¯¯¯¯Example 2:¯¯¯¯¯¯Input: MISMatch Date of Birth and profile/occupation. customer name is MUHAMAD SYAZWANUDDIN BIN SAH'